In [ ]:
import requests
import pandas as pd
import plotly.express as px
from pprint import pprint

weather_api_url = "https://api.weather.gov"
weather_api_headers = {"User-Agent": "(MastaPegsWeatherTest, mastapegs01@gmail.com)"}

# coordinates for Johnston RI
latitude = "41.8268"
longitude = "-71.5130"


def get_weather(path: str, use_full_url=False):
    r = requests.get(
        path if use_full_url else f"{weather_api_url}/{path}",
        headers=weather_api_headers,
    )

    return r.json()

# Weather API

Source Documentation: https://www.weather.gov/documentation/services-web-api

## Authentication

A User Agent is required to identify your application. This string can be anything, and the more unique to your application the less likely it will be affected by a security event. If you include contact information (website or email), we can contact you if your string is associated to a security event. This will be replaced with an API key in the future.

`User-Agent: (myweatherapp.com, contact@myweatherapp.com)`

In [ ]:
weather_data = get_weather("")

pprint(weather_data)

## Local Coordinates

02919/Coordinates
41.8268° N, 71.5130° W

`https://api.weather.gov/points/{latitude},{longitude}`
For example: https://api.weather.gov/points/39.7456,-97.0892

In [ ]:
data = get_weather(f"/points/{latitude},{longitude}")
hourly_forcast_url = data["properties"]["forecastHourly"]

#

hourly_forcast_data = get_weather(hourly_forcast_url, use_full_url=True)
hourly_forcast_periods = hourly_forcast_data["properties"]["periods"]

pprint(hourly_forcast_periods[1])
print("")

periods_useful_data = [
    {
        "startTime": period["startTime"],
        "temperature": period["temperature"],
        "relativeHumidity": period["relativeHumidity"]["value"],
        "probabilityOfPrecipitation": period["probabilityOfPrecipitation"]["value"],
        "windSpeed": int(str(period["windSpeed"]).split(" ")[0])
    }
    for period in hourly_forcast_periods
]

df = pd.DataFrame(periods_useful_data)
df["startTime"] = pd.to_datetime(df["startTime"])

print(df.dtypes)

df

## Plotting the useful data

In [ ]:
fig = px.scatter(
    df,
    x="startTime",
    y="temperature",
    color="probabilityOfPrecipitation",
    size="windSpeed",
    title="Johnston/Smithfield Hourly Weather Forcast",
    trendline="lowess",
    trendline_color_override="black",
    trendline_options=dict(frac=0.15),
)
fig.show()